In [2]:
# Import libraries.
from dataclasses import replace
import pandas as pd
import phoenix as px


In [10]:
import numpy as np
OUTDIR="/data/bacteria/experiments/autoencoders"
KMER=6
xq = np.load(f"{OUTDIR}/{KMER}mer/faiss-embeddings/query_embeddings.npy").astype("float32")


In [12]:
import json


In [40]:
df_labels = pd.read_csv("/data/bacteria/experiments/autoencoders/6mer/test/closest_species.tsv",sep="\t", usecols=["GT","consensus"])

In [51]:
from pathlib import Path
from collections import namedtuple

Phoenix=namedtuple("Phoenix",["id","path_img","link_img","label","vector_img"])
data_phoenix = []

with open(f"{OUTDIR}/{KMER}mer/faiss-embeddings/query_embeddings.json") as fp:
    id_query = json.load(fp)
    id_query = {int(k): v for k,v in id_query.items()}

REPLACE="/data/bacteria/experiments/autoencoders"
for idx, path_npy in id_query.items():
    _id = Path(path_npy).stem
    path_img = path_npy.replace("fcgr","img").replace(".npy",".jpeg")
    link_img = path_img.replace(REPLACE,"http://localhost:6007") # I choose the port 6007, and I need to run `python -m http.server 6007` in the REPLACE directory
    label = Path(path_img).parent.stem.split("__")[0]
    embedding = xq[idx]
    data_phoenix.append(
        Phoenix(_id, path_img, link_img, label, embedding)
    )
    

In [52]:
train_df = pd.concat([pd.DataFrame(data_phoenix), df_labels],axis=1)
train_df.head()

,id,path_img,link_img,label,vector_img,GT,consensus
0,SAMD00109104,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/pseudomonas_aer...,pseudomonas_aeruginosa,"[301.30157, 0.0, 143.3061, 299.01096, 0.0, 0.0...",pseudomonas_aeruginosa,pseudomonas_aeruginosa
1,SAMN07421883,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/haemophilus_inf...,haemophilus_influenzae,"[169.97614, 0.0, 141.93373, 0.0, 0.0, 0.0, 0.0...",haemophilus_influenzae,haemophilus_influenzae
2,SAMEA4427151,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/enterobacter_ho...,enterobacter_hormaechei,"[0.0, 0.0, 238.53975, 0.0, 0.0, 0.0, 0.0, 416....",enterobacter_hormaechei,enterobacter_hormaeche
3,SAMEA2239764,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/clostridioides_...,clostridioides_difficile,"[285.54553, 0.0, 0.0, 0.0, 1426.2489, 184.0323...",clostridioides_difficile,clostridioides_diffici
4,SAMN01047745,/data/bacteria/experiments/autoencoders/6mer/i...,http://localhost:6007/6mer/img/brucella_suis__...,brucella_suis,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",brucella_suis,brucella_suis


In [53]:
# Define schemas that tell Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.
train_schema = px.Schema(
    prediction_id_column_name="id",
    # timestamp_column_name="prediction_ts",
    prediction_label_column_name="consensus",
    actual_label_column_name="GT",
    embedding_feature_column_names={
        "image_embedding": px.EmbeddingColumnNames(
            vector_column_name="vector_img",
            link_to_data_column_name="link_img",
        ),
    },
)

In [3]:
# # Download curated datasets and load them into pandas DataFrames.
# train_df = pd.read_parquet(
#     "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/cv/human-actions/human_actions_training.parquet"
# )

In [5]:
# prod_df = pd.read_parquet(
#     "https://storage.googleapis.com/arize-assets/phoenix/datasets/unstructured/cv/human-actions/human_actions_production.parquet"
# )

In [54]:
# Define schemas that tell Phoenix which columns of your DataFrames correspond to features, predictions, actuals (i.e., ground truth), embeddings, etc.
# train_schema = px.Schema(
#     prediction_id_column_name="prediction_id",
#     timestamp_column_name="prediction_ts",
#     prediction_label_column_name="predicted_action",
#     actual_label_column_name="actual_action",
#     embedding_feature_column_names={
#         "image_embedding": px.EmbeddingColumnNames(
#             vector_column_name="image_vector",
#             link_to_data_column_name="url",
#         ),
#     },
# )
# prod_schema = replace(train_schema, actual_label_column_name=None)

# Define your production and training datasets.
# prod_ds = px.Dataset(prod_df, prod_schema)
train_ds = px.Dataset(train_df, train_schema)

# Launch Phoenix.
session = px.launch_app(
    # prod_ds, 
    train_ds
    )

# View the Phoenix UI in the browser
# session.url
px.active_session().view()

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://127.0.0.1:6006/


/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:422: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  prediction_id=dataset[PREDICTION_ID][row_id],
/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:423: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  link_to_data=dataset[self.dimension.link_to_data][row_id],
/home/avila/micromamba/envs/phoenix/lib/python3.11/site-packages/phoenix/server/api/types/EmbeddingDimension.py:424: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. 